# Importing Libraries

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
!pip install imperio
import imperio
import tensorflow as tf
import seaborn as sns

In [ ]:
!pip install fuzzywuzzy
from fuzzywuzzy import fuzz, process

# Import data

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/Organizatii/Sigmoid/Padawan Learning/Exam/data2.csv')

In [ ]:
df

# Analyzing

In [ ]:
df.info()

There could be observed that Some coulmns (Population, GDP, Hepatitis B) has significant number of NaN data

In [ ]:
plt.figure(figsize=(14,12))
sns.heatmap(df.corr())
plt.show()

"thinness 5-9" is repeating in "thinness 1-19"

In [ ]:
status_groupby = df.groupby('Status', as_index=False)
status_groupby['Life expectancy '].describe()

From the above table there could be possible to extract following information:


1.   There are less Developed countries than in the process. 
2.   The variation of life expectancy in the countries that are in process of developing  is bigger. 



In [ ]:
# Get random data of a country
data_Afganistan = df[df['Country'] == 'Afghanistan']

# Plot GDP per years
plt.plot(data_Afganistan['Year'].astype(int), data_Afganistan['GDP'])

In [ ]:
# Plot population per years
plt.plot(data_Afganistan['Year'].astype(int), data_Afganistan['Population'])

The real population of Afhanistan in 2015 was 33 millions (source: Wikipedia).
This means that data in the following years [2014, 2010,2008,2006, 2005,2003,2001,2000] is wrong, because it has an enormous deviation.

From the above graph we can see that there is a problem in data. The person who entered the data made mistakes in entering the data, sometimes not adding a digit to the number, for example instead of 36043432, he entered only 3604343, and respectively the number is 10 times smaller than the initial one, as a result it creates enormous disturbances in dataset.

### Plot populatation of multiple countries

In [ ]:
# Plot Population of 26 countries

# Get random 26 countries
countries_to_analyze = df['Country'].sample(n=26).values

# Per each country plot population and outliers.

x = y = 0 # coordinates for plot
fig, axs = plt.subplots(13, 2, figsize=(12, 40), constrained_layout=True)

for country in countries_to_analyze:
    # Get all 15 rows (15 years) for selected country
    df_country = df[df['Country'] == country]

    # Ploting data
    axs[x,y].set_title(f'Population of {country}')
    axs[x,y].scatter(df_country['Year'].astype(int), df_country['Population'])

    # Changing coordinates for ploting on next graph
    if y == 0:
        y = 1
    else:
        y = 0
        x += 1

We can see that the problem lies in most countries.
For example, in Greece the population in 2015 was 10.86 million people, but in the graph only 1 million is represented (10 times smaller). Respectively, only one year (2010) has the correct number, the rest have the mistake.

### Plot GDP of multiple countries

In [ ]:
# Plot GDP of 26 countries

# Get random 26 countries
countries_to_analyze = df['Country'].sample(n=26).values

# Per each country plot population and outliers.

x = y = 0 # coordinates for plot
fig, axs = plt.subplots(13, 2, figsize=(12, 40), constrained_layout=True)

for country in countries_to_analyze:
    # Get all 15 rows (15 years) for selected country
    df_country = df[df['Country'] == country]

    # Ploting data
    axs[x,y].set_title(f'GDP of {country}')
    axs[x,y].scatter(df_country['Year'].astype(int), df_country['GDP'])

    # Changing coordinates for ploting on next graph
    
    if y == 0:
        y = 1
    else:
        y = 0
        x += 1

# Preprocessing

In [ ]:
df.columns

In [ ]:
# Remove white spaces in column names
df.columns = df.columns.str.strip()

In [ ]:
# drop thinness 5-9
df.drop(['thinness 5-9 years'], axis=1, inplace=True)

## Dealing with non-valid data  (GDP and Population)

#### Population 

In [ ]:
# Import a new csv file from World Bank Data
population_df = pd.read_csv('/content/drive/MyDrive/Organizatii/Sigmoid/Padawan Learning/Exam/World_Bank_Data_Population.csv', error_bad_lines=False, header=2)
population_df

From the above (new dataset) we can see that it contains total population of each county per years. 

Now we need to extract data from the new dataset and assign it to the previous one

In [ ]:
population_df.columns

In [ ]:
# Drop Non-Usable columns
population_droped_df = population_df.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 66'], axis=1)

# Drop Age-Columns that are out of our scope
population_droped_df = population_droped_df.drop(
    [i for i in population_droped_df.columns[1:] if int(i) < 2000 or int(i) > 2015],
    axis=1)

population_droped_df

Now we need to create a new data set, which will contain the first row - the name of the country and the second row - the year and the third row the population for that year.

In [ ]:
new_df_population = population_droped_df.melt(id_vars=['Country Name'], value_vars=population_droped_df.columns[1:], var_name='Year', value_name='Population')
new_df_population = new_df_population.rename(columns={'Country Name': 'Country'})
new_df_population['Year'] = new_df_population['Year'].astype(np.int64)

There are two datasets (from WH and WDB). They can contain name of the same country but written differently. Ex: "USA" and "United States of America" etc.

---
For handling this problem we will eliminate "noisy words" and will search for names that look like most similar between themselves. 

In [ ]:
map_countries = {}

# Creating bag of words for names of the countries
bag_words = ['of', 'republic', 'islam', 'democratic', "peoples", 'arab', 'the']

def clear_words(name):
    # Clear from non-alpha symbols
    name = ''.join([i for i in name if i.isalpha() or i == ' '])
    return ' '.join(i for i in name.split() if i.lower() not in bag_words)

new_df_population['Country'] = new_df_population['Country'].apply(lambda x: clear_words(x))
df['Country'] = df['Country'].apply(lambda x: clear_words(x))


# Define a function that takes in a country name and returns the closest match
def match_name(name, list_of_names, min_score=0):
    match = process.extractOne(name, list_of_names, scorer=fuzz.token_sort_ratio)
    if match[1] > min_score:
        return match[0]
    else:
        return ""

# Create a new column in each DataFrame with matched country namesmatch_name
for country in df['Country'].unique():
    matched_name = match_name(country, new_df_population['Country'].unique())
    map_countries[matched_name] = country

In [ ]:
# Create a new column in each DataFrame with matched country names
new_df_population['Matched Country'] = new_df_population['Country'].apply(lambda x: map_countries.get(x))

# Merge the DataFrames on the matched country names column
merged_df = pd.merge(df, new_df_population, left_on=['Country', 'Year'], right_on=['Matched Country', 'Year'], how='left')

# update the population column
merged_df['Population'] = merged_df['Population_y']
merged_df['Population'] = np.where(merged_df['Population'].isnull(), merged_df['Population_x'], merged_df['Population_y'])

merged_df = merged_df.rename(columns = {'Country_x' : 'Country'})
df = merged_df.drop(columns=['Population_x', 'Population_y', 'Matched Country', 'Country_y'])

Ploting new data

In [ ]:
# Get random 26 countries
countries_to_analyze = merged_df['Country'].sample(n=26).values

# Per each country plot population.
x = y = 0 # coordinates for plot
fig, axs = plt.subplots(13, 2, figsize=(12, 40), constrained_layout=True)

for country in countries_to_analyze:
    # Get all 15 rows (15 years) for selected country
    df_country = merged_df[merged_df['Country'] == country]

    # Ploting data
    axs[x,y].set_title(f'Population of {country}')
    axs[x,y].scatter(df_country['Year'].astype(int), df_country['Population'])

    # Changing coordinates for ploting on next graph
    if y == 0:
        y = 1
    else:
        y = 0
        x += 1

#### GDP

In [ ]:
gdp_df = pd.read_csv('/content/drive/MyDrive/Organizatii/Sigmoid/Padawan Learning/Exam/World_Bank_Data_GDP.csv', error_bad_lines=False, header=2)
gdp_df

In [ ]:
# Get columns 
gdp_df.columns

Now we can see that the graph above is smoother

In [ ]:
# Drop Non-Usable columns
gdp_droped_df = gdp_df.drop(['Country Code', 'Indicator Name', 'Indicator Code', 'Unnamed: 66'], axis=1)

# Drop Age-Columns that are out of our scope
gdp_droped_df = gdp_droped_df.drop(
    [i for i in gdp_droped_df.columns[1:] if int(i) < 2000 or int(i) > 2015],
    axis=1)

gdp_droped_df

Need to create a new dataset, that will contains first row - name of the country, and second row - year, and third row population per that year.

In [ ]:
new_df_gdp = gdp_droped_df.melt(id_vars=['Country Name'], value_vars=gdp_droped_df.columns[1:], var_name='Year', value_name='GDP')
new_df_gdp = new_df_gdp.rename(columns={'Country Name': 'Country'})
new_df_gdp['Year'] = new_df_gdp['Year'].astype(np.int64)

There are two datasets. They can contain name of the same country but written differently. Ex: "USA" and "United States of America" etc.

---
For handling this problem we will eliminate "noisy words" and will search for names that look like most similar between themselves. 

In [ ]:
map_countries = {}

# Creating bag of words for names of the countries
bag_words = ['of', 'republic', 'islam', 'democratic', "peoples", 'arab', 'the']

def clear_words(name):
    # Clear from non-alpha symbols
    name = ''.join([i for i in name if i.isalpha() or i == ' '])
    return ' '.join(i for i in name.split() if i.lower() not in bag_words)

new_df_gdp['Country'] = new_df_gdp['Country'].apply(lambda x: clear_words(x))
df['Country'] = df['Country'].apply(lambda x: clear_words(x))


# Define a function that takes in a country name and returns the closest match
def match_name(name, list_of_names, min_score=0):
    match = process.extractOne(name, list_of_names, scorer=fuzz.token_sort_ratio)
    if match[1] > min_score:
        return match[0]
    else:
        return ""

# Create a new column in each DataFrame with matched country namesmatch_name
for country in df['Country'].unique():
    matched_name = match_name(country, new_df_gdp['Country'].unique())
    map_countries[matched_name] = country

In [ ]:
# Create a new column in each DataFrame with matched country names
new_df_gdp['Matched Country'] = new_df_gdp['Country'].apply(lambda x: map_countries.get(x))

# Merge the DataFrames on the matched country names column
merged_df = pd.merge(df, new_df_gdp, left_on=['Country', 'Year'], right_on=['Matched Country', 'Year'], how='left')

# update the GDP column
merged_df['GDP'] = merged_df['GDP_y']
merged_df['GDP'] = np.where(merged_df['GDP'].isnull(), merged_df['GDP_x'], merged_df['GDP_y'])

merged_df = merged_df.rename(columns = {'Country_x' : 'Country'})
df = merged_df.drop(columns=['GDP_x', 'GDP_y', 'Matched Country', 'Country_y'])

Ploting new data

In [ ]:
# Get random 26 countries
countries_to_analyze = merged_df['Country'].sample(n=26).values

# Per each country plot GDP.

x = y = 0 # coordinates for plot
fig, axs = plt.subplots(13, 2, figsize=(12, 40), constrained_layout=True)

for country in countries_to_analyze:
    # Get all 15 rows (15 years) for selected country
    df_country = merged_df[merged_df['Country'] == country]

    # Ploting data
    axs[x,y].set_title(f'GDP of {country}')
    axs[x,y].scatter(df_country['Year'].astype(int), df_country['GDP'])

    # Changing coordinates for ploting on next graph
    if y == 0:
        y = 1
    else:
        y = 0
        x += 1

## Dealing with categorical data

In [ ]:
# One hot encoding "Status" column
df['Status'] = pd.get_dummies(df['Status'].values, drop_first=True).values

In [ ]:
df

## Spliting in Test and train

In [ ]:
y = df['Life expectancy']
X = df.drop(['Life expectancy'], axis=1)

# Split in train/test

from sklearn.model_selection import train_test_split
training_data, testing_data = train_test_split(df, test_size=0.3, random_state=25)

In [ ]:
# For better accuracy delete nan values from testing data
testing_data = testing_data.dropna()

## Dealing with NaN values

In [ ]:
training_data.isna().sum()

In [ ]:
training_data[training_data['Hepatitis B'].isna()]

There is a problem with NaN values at "Hepatitis B " columns. Practically 25% of our dataset does not have this data.


---

From the above cell we can conclude that there is a big variation between minimum and maximum value. That's why we will get mean value based on country Status and Life Expectancy of others countries

In [ ]:
# Filling missing data by grouping by country and filling with mean using previous/future values
grouped =  training_data.groupby(["Country", "Year"])

# Fill the missing values  with the calculated mean for the same country
training_data["Alcohol"] =  training_data.groupby(["Country"])["Alcohol"].apply(lambda x: x.fillna(x.mean()))
# Fill the missing values in the 'Total expenditure' column with the calculated mean
training_data['Total expenditure'] =  training_data.groupby(["Country"])['Total expenditure'].apply(lambda x: x.fillna(x.mean()))
training_data['GDP'] =  training_data.groupby(["Country"])['GDP'].apply(lambda x: x.fillna(x.mean()))
training_data['Schooling'] =  training_data.groupby(["Country"])['Schooling'].apply(lambda x: x.fillna(x.mean()))

In [ ]:
training_data['Life expectancy group'] = pd.cut( training_data['Life expectancy'], bins=[0,30,35,40,45,50,55,60, 65, 70, 75, 80, 85, 90, 100], labels=['0-30','30-35','35-40','40-45','45-50','50-55','55-60','60-65', '65-70', '70-75', '75-80', '80-85', '85-90', '90-100'])

# Group the dataframe by "Status" and "Life expectancy group"
grouped =  training_data.groupby(['Status', 'Life expectancy group'])

# Fill NaN values in columns with mean of the group
training_data['Total expenditure'] = grouped['Total expenditure'].transform(lambda x: x.fillna(x.mean()))
training_data['Diphtheria'] = grouped['Diphtheria'].transform(lambda x: x.fillna(x.mean()))
training_data['thinness  1-19 years'] = grouped['thinness  1-19 years'].transform(lambda x: x.fillna(x.mean()))
training_data['Income composition of resources'] = grouped['Income composition of resources'].transform(lambda x: x.fillna(x.mean()))
training_data['Schooling'] = grouped['Schooling'].transform(lambda x: x.fillna(x.mean()))
training_data['BMI'] = grouped['BMI'].transform(lambda x: x.fillna(x.mean()))
training_data['Polio'] = grouped['Polio'].transform(lambda x: x.fillna(x.mean()))
training_data['Hepatitis B'] = grouped['Hepatitis B'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
training_data[training_data['Hepatitis B'].isna()]

We can see that majority of Hepatitis B  NaN value dissapeared, but there also remained 8 examples

We can see that rows that have missing value in Hepatitics B column have also missing value in others very important columns (Life expectancy, Adult Mortality, etc.)


In [ ]:
training_data =  training_data.dropna(subset=['Hepatitis B', 'Diphtheria'])
training_data["Alcohol"] =  training_data["Alcohol"].fillna(0)
training_data.isna().sum()

In [ ]:
# Drop "Life expectancy group"
training_data.drop(['Life expectancy group'], axis=1, inplace=True)

In [ ]:
# Drop Country column
training_data = training_data.drop(['Country'], axis=1)
testing_data = testing_data.drop(['Country'], axis=1)

## Separating X and y

In [ ]:
y_train = training_data['Life expectancy'].values
X_train = training_data.drop(['Life expectancy'], axis=1)

y_test = testing_data['Life expectancy'].values
X_test = testing_data.drop(['Life expectancy'], axis=1)

## Feature Engineering

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaller = MinMaxScaler()

In [ ]:
# Fit transform data
scaller.fit(X_train)

# Transform data
X_train = scaller.transform(X_train)
X_test= scaller.transform(X_test)

# Training model

In [ ]:
!pip install lazypredict
from lazypredict.Supervised import LazyRegressor

In [ ]:
reg = LazyRegressor(predictions=True)
models, predictions = reg.fit(X_train, X_test, y_train, y_test)
models

 90%|█████████ | 38/42 [03:54<00:43, 10.78s/it]

[21:48:36] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


100%|██████████| 42/42 [03:55<00:00,  5.60s/it]


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,0.96,0.97,1.56,1.01
LGBMRegressor,0.96,0.96,1.72,0.29
RandomForestRegressor,0.96,0.96,1.72,1.99
BaggingRegressor,0.95,0.95,1.81,0.43
HistGradientBoostingRegressor,0.95,0.95,1.82,6.97
XGBRegressor,0.93,0.94,2.12,0.33
GradientBoostingRegressor,0.93,0.94,2.13,0.82
DecisionTreeRegressor,0.90,0.90,2.59,0.08
ExtraTreeRegressor,0.89,0.90,2.70,0.03
